## Build your first Agent with memory ft. MCP

<img src="assets/agent-crashcourse2.gif" width="400" height="300">

In [34]:
from crewai import Crew, Process, Task, Agent
from dotenv import load_dotenv

load_dotenv()

True

### Setup a local LLM

In [35]:
from crewai import LLM

llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434"
)

### Simple QA crew

In [43]:
virtual_assistant = Agent(
    role="Virtual Assistant",
    goal="Help the user with their questions",
    backstory="You are an agent that can help the user with their questions.",
    allow_delegation=False,
    llm=llm
)

assistant_task = Task(
    description="Provide an answer to the user's {query}.",
    agent=virtual_assistant,
    expected_output="A concise answer to the user's query.",
)

crew = Crew(
    agents=[virtual_assistant],
    tasks=[assistant_task],
    verbose=True,
)

result = crew.kickoff(inputs={"query": "What is the capital of Spain?"})
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 205863a2-c452-466a-82f4-c6c3f3620ea7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Virtual Assistant                                                                                       │
│                                                                                                                 │
│  Task: Provide an answer to the user's What is the capital of Spain?.                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Virtual Assistant                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The capital of Spain is Madrid.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4e42aeb2-594b-4018-9890-5e061311221f                                                                     │
│  Agent: Virtual Assistant                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 205863a2-c452-466a-82f4-c6c3f3620ea7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The capital of Spain is Madrid.                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The capital of Spain is Madrid.


#### Tracing and observability

In [42]:
import opik

opik.configure(use_local=False)

from opik.integrations.crewai import track_crewai

track_crewai(project_name="crewai-integration-demo")

OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".
OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/akshay/.opik.config


### Web Search tool

In [ ]:
import os
print(os.getenv("LINKUP_API_KEY"))


In [44]:
from pydantic import BaseModel, Field
from linkup import LinkupClient
from typing import Type
from crewai.tools import BaseTool
import os

class LinkUpSearchInput(BaseModel):
    """Input schema for LinkUp Search Tool."""
    query: str = Field(description="The search query to perform")
    depth: str = Field(default="standard",
                       description="Depth of search: 'standard' or 'deep'")
    output_type: str = Field(
        default="searchResults", description="Output type: 'searchResults', 'sourcedAnswer', or 'structured'")


class LinkUpSearchTool(BaseTool):
    name: str = "LinkUp Search"
    description: str = "Search the web for information using LinkUp and return comprehensive results"
    args_schema: Type[BaseModel] = LinkUpSearchInput

    def __init__(self):
        super().__init__()

    def _run(self, query: str, depth: str = "standard", output_type: str = "searchResults") -> str:
        """Execute LinkUp search and return results."""
        try:
            # Initialize LinkUp client with API key from environment variables
            linkup_client = LinkupClient(api_key=os.getenv("LINKUP_API_KEY"))

            # Perform search
            search_response = linkup_client.search(
                query=query,
                depth=depth,
                output_type=output_type
            )

            return str(search_response)
        except Exception as e:
            return f"Error occurred while searching: {str(e)}"

### Crew with tools

In [45]:
web_search_agent = Agent(
    role="Web Search Agent",
    goal="Search the web for information",
    backstory="You are an agent that can search the web for information.",
    allow_delegation=False,
    tools=[LinkUpSearchTool()],
)

web_search_task = Task(
    description="Search the web for information about {query}.",
    agent=web_search_agent,
    expected_output="A concise answer to the user's query based on the search results.",
)

crew = Crew(
    agents=[web_search_agent],
    tasks=[web_search_task],
    verbose=True,
)

result = crew.kickoff(inputs={"query": "Give me a summary of what is happening in Wimbeldon 2025?"})
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 779faaeb-5a67-41e0-9f33-35282486146b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Agent                                                                                        │
│                                                                                                                 │
│  Task: Search the web for information about Give me a summary of what is happening in Wimbeldon 2025?.          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/akshay/Eigen/ai-engineering-hub/agent-with-mcp-memory/.venv/lib/python3.12/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Agent                                                                                        │
│                                                                                                                 │
│  Thought: Action: LinkUp Search                                                                                 │
│                                                                                                                 │
│  Using Tool: LinkUp Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"Wimbledon 2025 latest news\", \"depth\": \"standard\", \"output_type\": \"searchResults\"}"     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  results=[LinkupSearchTextResult(type='text', name='Schedule - The Championships, Wimbledon',                   │
│  url='https://www.wimbledon.com/en_GB/atoz/schedule.html', content='We cannot provide a description for this    │
│  page right now\nundefined'), LinkupSearchTextResult(type='text', name='Wimbledon 2025: Results & updates',     │
│  url='https://www.lta.org.uk/fan-zone/wimbledon-championships/news/wimbledon-2025-results-updates/',            │
│  content='Emma Raducanu, Cam Norrie &amp; Sonay Kartal Into The Third Round 🍓 | Wimbledon 2025 Highlights |    │
│  LTA · Draper, Evans &amp; Pinnington Jones Storm Into Round Two 🔥 | Wimbledon 2025 Highlights | LTA\nFollow   │
│  the results and draws for every British player competing at Wimbledon 2025.\nFind out everything you need to   │
│  know about The Championships, Wimbledon 2025 including key dates, draws, order of play, scores and who is set  │
│  to play.\nEmma Raducanu, Cam Norrie & Sonay Kartal Into The Third Round 🍓 | Wimbledon 2025 Highlights | LTA   │
│  · Draper, Evans & Pinnington Jones Storm Into Round Two 🔥 | Wimbledon 2025 Highlights | LTA\nRaducanu,        │
│  Boulter, Kartal, Fery, Harris, Norrie Win on Big Day for Brits | Wimbledon 2025 Highlights'),                  │
│  LinkupSearchTextResult(type='text', name='Wimbledon 2025: London weather results in multiple fan-related heat  │
│  delays, with Aryna Sabalenka stepping in to help - Yahoo Sports',                                              │
│  url='https://sports.yahoo.com/tennis/breaking-news/article/wimbledon-2025-london-weather-results-in-multiple-  │
│  fan-related-heat-delays-with-aryna-sabalenka-stepping-in-to-help-133351232.html', content="Sabalenka fought    │
│  back in the second set, taking it 6-4 to take the game to a final set. After a fantastic, back-and-forth       │
│  match, it was Anisimova who prevailed. She took down the No. 1 women&#x27;s player 6-4, 4-6, 6-4 to advance    │
│  to her first Wimbledon final.\nSabalenka fought back in the second set, taking it 6-4 to take the game to a    │
│  final set. After a fantastic, back-and-forth match, it was Anisimova who prevailed. She took down the No. 1    │
│  women's player 6-4, 4-6, 6-4...                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Wimbledon 2025 is underway at the All England Club in London, running from June 30 to July 13. The tournament  │
│  has experienced some weather-related heat delays affecting matches. In the women's singles, Amanda Anisimova   │
│  defeated top seed Aryna Sabalenka in a thrilling three-set match (6-4, 4-6, 6-4) to advance to her first       │
│  Wimbledon final, while Iga Swiatek also reached the final after a dominant win (6-2, 6-0) over Belinda         │
│  Bencic. This ensures a first-time women's Wimbledon champion in 2025, as defending champions have exited       │
│  early. On the men's side, Carlos Alcaraz, the two-time defending champion aiming for a three-peat, advanced    │
│  to the semifinals alongside Novak Djokovic, who will face Jannik Sinner in the other semifinal. Other notable  │
│  performances include American Taylor Fritz reaching the men's semifinals and British players like Cameron      │
│  Norrie having strong runs before being eliminated. The tournament has drawn a mix of sports stars,             │
│  celebrities, and royalty to the royal box. Wimbledon 2025 continues to deliver historic and exciting moments   │
│  with the finals imminent.                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8e9b17ee-4cbc-4000-8524-59c7364c18a7                                                                     │
│  Agent: Web Search Agent                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 779faaeb-5a67-41e0-9f33-35282486146b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Wimbledon 2025 is underway at the All England Club in London, running from June 30 to July 13.   │
│  The tournament has experienced some weather-related heat delays affecting matches. In the women's singles,     │
│  Amanda Anisimova defeated top seed Aryna Sabalenka in a thrilling three-set match (6-4, 4-6, 6-4) to advance   │
│  to her first Wimbledon final, while Iga Swiatek also reached the final after a dominant win (6-2, 6-0) over    │
│  Belinda Bencic. This ensures a first-time women's Wimbledon champion in 2025, as defending champions have      │
│  exited early. On the men's side, Carlos Alcaraz, the two-time defending champion aiming for a three-peat,      │
│  advanced to the semifinals alongside Novak Djokovic, who will face Jannik Sinner in the other semifinal.       │
│  Other notable performances include American Taylor Fritz reaching the men's semifinals and British players     │
│  like Cameron Norrie having strong runs before being eliminated. The tournament has drawn a mix of sports       │
│  stars, celebrities, and royalty to the royal box. Wimbledon 2025 continues to deliver historic and exciting    │
│  moments with the finals imminent.                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Wimbledon 2025 is underway at the All England Club in London, running from June 30 to July 13. The tournament has experienced some weather-related heat delays affecting matches. In the women's singles, Amanda Anisimova defeated top seed Aryna Sabalenka in a thrilling three-set match (6-4, 4-6, 6-4) to advance to her first Wimbledon final, while Iga Swiatek also reached the final after a dominant win (6-2, 6-0) over Belinda Bencic. This ensures a first-time women's Wimbledon champion in 2025, as defending champions have exited early. On the men's side, Carlos Alcaraz, the two-time defending champion aiming for a three-peat, advanced to the semifinals alongside Novak Djokovic, who will face Jannik Sinner in the other semifinal. Other notable performances include American Taylor Fritz reaching the men's semifinals and British players like Cameron Norrie having strong runs before being eliminated. The tournament has drawn a mix of sports stars, celebrities, and royalty to the royal box. Wi

### CrewAI meets MCP

Using MCP servers as tools with Crew Agents

In [46]:
from crewai import Agent
from crewai_tools import MCPServerAdapter


# 2. SSE Server:
server_params = {
    "url": "http://localhost:8080/sse",
    "transport": "sse"
}

with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")

Available tools: ['web_search']


/Users/akshay/Eigen/ai-engineering-hub/agent-with-mcp-memory/.venv/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'items', 'anyOf', 'enum', 'properties'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [47]:
# Example usage (uncomment and adapt once server_params is set):
with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")

    web_search_agent = Agent(
        role="Web Search Agent",
        goal="Search the web for information",
        backstory="You are an agent that can search the web for information.",
        allow_delegation=False,
        tools=[mcp_tools["web_search"]],
    )

    web_search_task = Task(
        description="Search the web for information about {query}.",
        agent=web_search_agent,
        expected_output="A concise answer to the user's query based on the search results.",
    )

    crew = Crew(
        agents=[web_search_agent],
        tasks=[web_search_task],
        verbose=True,
    )

    result = crew.kickoff(inputs={"query": "Give me a summary of what is happening in Wimbledon 2025?"})
    print(result)

Available tools: ['web_search']


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 13cf4d2e-3e8e-40fe-9918-8446032896ce                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Agent                                                                                        │
│                                                                                                                 │
│  Task: Search the web for information about Give me a summary of what is happening in Wimbledon 2025?.          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/akshay/Eigen/ai-engineering-hub/agent-with-mcp-memory/.venv/lib/python3.12/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Agent                                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to search for current information about what is happening in Wimbledon 2025 to        │
│  provide an accurate and detailed summary.                                                                      │
│                                                                                                                 │
│  Using Tool: web_search                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  answer="Wimbledon 2025 is underway from June 30 to July 13. Key updates include:\n\n- Women's semifinals took  │
│  place on July 10, with Amanda Anisimova defeating top seed Aryna Sabalenka 6-4, 4-6, 6-4 to reach her first    │
│  Wimbledon final. Iga Swiatek also advanced to the final after a 6-2, 6-0 win over Belinda Bencic.\n- Men's     │
│  semifinals are scheduled for July 11, featuring Novak Djokovic vs. Jannik Sinner and Carlos Alcaraz vs.        │
│  Taylor Fritz.\n- Several British players, including Emma Raducanu, Cameron Norrie, and Sonay Kartal, have      │
│  progressed to later rounds.\n- The tournament has seen many upsets, with only nine of the top 32 seeds in      │
│  both men's and women's draws reaching the fourth round.\n- Weather has caused some heat-related delays,        │
│  notably during the Sabalenka-Anisimova match.\n- Celebrity and royal attendance continues, with notable        │
│  figures appearing in the Royal Box.\n- Tickets remain available primarily through debenture holders, with      │
│  prices varying by court and match day.\n\nFor live scores, draws, and detailed schedules, official Wimbledon   │
│  and LTA websites provide continuous updates." sources=[LinkupSource(name='Schedule - The Championships,        │
│  Wimbledon', url='https://www.wimbledon.com/en_GB/atoz/schedule.html', snippet='We cannot provide a             │
│  description for this page right now\nundefined'), LinkupSource(name='Wimbledon 2025: Results & updates',       │
│  url='https://www.lta.org.uk/fan-zone/wimbledon-championships/news/wimbledon-2025-results-updates/',            │
│  snippet='Emma Raducanu, Cam Norrie &amp; Sonay Kartal Into The Third Round 🍓 | Wimbledon 2025 Highlights |    │
│  LTA · Draper, Evans &amp; Pinnington Jones Storm Into Round Two 🔥 | Wimbledon 2025 Highlights | LTA\nFollow   │
│  the results and draws for every British player competing at Wimbledon 2025.\nFind out everything you need to   │
│  know about The Championships, Wimbledon 2025 including key dates, draws, order of play, scores and who is set  │
│  to play.\nEmma Raducanu, Cam Norrie & Sonay Kartal Into The Third Round 🍓 | Wimbledon 2025 ...                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Search Agent                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Wimbledon 2025 is currently taking place from June 30 to July 13. The women's semifinals were held on July     │
│  10, where Amanda Anisimova defeated top seed Aryna Sabalenka 6-4, 4-6, 6-4 to reach her first Wimbledon        │
│  final, and Iga Swiatek advanced after a 6-2, 6-0 win over Belinda Bencic. The men's semifinals are scheduled   │
│  for July 11, featuring Novak Djokovic vs. Jannik Sinner and Carlos Alcaraz vs. Taylor Fritz. Several British   │
│  players, including Emma Raducanu, Cameron Norrie, and Sonay Kartal, have progressed to later rounds. The       │
│  tournament has seen many upsets, with only nine of the top 32 seeds in both the men's and women's draws        │
│  reaching the fourth round. Weather delays due to heat have occurred, especially during the                     │
│  Sabalenka-Anisimova match. Celebrity and royal attendance continues, with notable figures appearing in the     │
│  Royal Box. Tickets mostly remain available through debenture holders, with prices varying. For live scores,    │
│  draws, and detailed schedules, the official Wimbledon and LTA websites provide continuous updates.             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ef7546f4-67ef-4759-a2ca-f1edcb2e8980                                                                     │
│  Agent: Web Search Agent                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 13cf4d2e-3e8e-40fe-9918-8446032896ce                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Wimbledon 2025 is currently taking place from June 30 to July 13. The women's semifinals were    │
│  held on July 10, where Amanda Anisimova defeated top seed Aryna Sabalenka 6-4, 4-6, 6-4 to reach her first     │
│  Wimbledon final, and Iga Swiatek advanced after a 6-2, 6-0 win over Belinda Bencic. The men's semifinals are   │
│  scheduled for July 11, featuring Novak Djokovic vs. Jannik Sinner and Carlos Alcaraz vs. Taylor Fritz.         │
│  Several British players, including Emma Raducanu, Cameron Norrie, and Sonay Kartal, have progressed to later   │
│  rounds. The tournament has seen many upsets, with only nine of the top 32 seeds in both the men's and women's  │
│  draws reaching the fourth round. Weather delays due to heat have occurred, especially during the               │
│  Sabalenka-Anisimova match. Celebrity and royal attendance continues, with notable figures appearing in the     │
│  Royal Box. Tickets mostly remain available through debenture holders, with prices varying. For live scores,    │
│  draws, and detailed schedules, the official Wimbledon and LTA websites provide continuous updates.             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Wimbledon 2025 is currently taking place from June 30 to July 13. The women's semifinals were held on July 10, where Amanda Anisimova defeated top seed Aryna Sabalenka 6-4, 4-6, 6-4 to reach her first Wimbledon final, and Iga Swiatek advanced after a 6-2, 6-0 win over Belinda Bencic. The men's semifinals are scheduled for July 11, featuring Novak Djokovic vs. Jannik Sinner and Carlos Alcaraz vs. Taylor Fritz. Several British players, including Emma Raducanu, Cameron Norrie, and Sonay Kartal, have progressed to later rounds. The tournament has seen many upsets, with only nine of the top 32 seeds in both the men's and women's draws reaching the fourth round. Weather delays due to heat have occurred, especially during the Sabalenka-Anisimova match. Celebrity and royal attendance continues, with notable figures appearing in the Royal Box. Tickets mostly remain available through debenture holders, with prices varying. For live scores, draws, and detailed schedules, the official Wimbledon an

### Knowledge graph based human like memory for Agents

<img src="assets/zep-memory.gif" width="400" height="400">

### Agents with knowledge graph memory


<img src="assets/agent-crashcourse1.gif" width="400" height="400">

In [48]:
from crewai import Agent, Task, Crew

from crewai_tools import MCPServerAdapter

# MCP configuration for websearch and memory MCP servers
server_params = [
    {
        "url": "http://localhost:8000/sse",
        "transport": "sse"
    },
    {
    "url": "http://localhost:8080/sse",
    "transport": "sse"
    }
  ]

# Create agents under the MCP context manager
with MCPServerAdapter(server_params) as mcp_tools:
    # Assistant Agent
    assistant_agent = Agent(
        role="Helpful Assistant",
        goal="Provide accurate and helpful responses to user queries",
        backstory="I am a helpful AI assistant that can search the web when needed to provide accurate information.",
        allow_delegation=False,
        tools=[mcp_tools["web_search"]]
    )

    # Memory Manager Agent  
    memory_manager = Agent(
        role="Memory Manager",
        goal="Store and manage conversation history",
        backstory="I am responsible for maintaining the conversation memory by storing relevant information.",
        allow_delegation=False,
        tools=[mcp_tools["add_memory"]]
    )

    # Response Generator Agent
    response_generator = Agent(
        role="Response Generator",
        goal="Generate coherent responses using memory context",
        backstory="I analyze memory nodes to generate contextually relevant responses.",
        allow_delegation=False,
        tools=[mcp_tools["search_memory_nodes"]]
    )

    # Define tasks
    assistant_task = Task(
        description="Process the user query '{query}' and provide a helpful response. Search the web if needed.",
        agent=assistant_agent,
        expected_output="A detailed response addressing the user's query"
    )

    memory_task = Task(
        description="Store the query '{query}' and response in memory for future reference",
        agent=memory_manager,
        expected_output="Confirmation of memory storage"
    )

    response_gen_task = Task(
        description="Generate a response for '{query}' using relevant information from memory",
        agent=response_generator,
        expected_output="A coherent response incorporating context from memory"
    )

    # Create crew with all agents and tasks
    crew = Crew(
        agents=[assistant_agent, memory_manager, response_generator],
        tasks=[assistant_task, memory_task, response_gen_task],
        verbose=True
    )

    # Example usage
    result = crew.kickoff(
        inputs={
            "query": "What is happening in FIFA club world cup 2025?",
        }
    )
    print(result)


/Users/akshay/Eigen/ai-engineering-hub/agent-with-mcp-memory/.venv/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'items', 'anyOf', 'enum', 'properties'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 40ed7443-2bf9-4a19-b9ca-b9caf3d7280a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Helpful Assistant                                                                                       │
│                                                                                                                 │
│  Task: Process the user query 'What is happening in FIFA club world cup 2025?' and provide a helpful response.  │
│  Search the web if needed.                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/akshay/Eigen/ai-engineering-hub/agent-with-mcp-memory/.venv/lib/python3.12/site-packages/rich/live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Helpful Assistant                                                                                       │
│                                                                                                                 │
│  Thought: Thought: To provide a detailed and accurate response about what is happening in the FIFA Club World   │
│  Cup 2025, I need to gather the latest information about the event. I will search the web for updates, news,    │
│  and details about the FIFA Club World Cup 2025.                                                                │
│                                                                                                                 │
│  Using Tool: web_search                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Response Generator                                                                                      │
│                                                                                                                 │
│  Task: Generate a response for 'What is happening in FIFA club world cup 2025?' using relevant information      │
│  from memory                                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Response Generator                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The 2025 FIFA Club World Cup is currently taking place in the United States from June 14 to July 13, 2025.     │
│  This edition of the tournament features a significant change with an expanded format including 32 clubs        │
│  competing in 12 venues across 11 US cities. It is designed as a prelude to the 2026 FIFA World Cup.            │
│                                                                                                                 │
│  Key details and happenings in the tournament include:                                                          │
│                                                                                                                 │
│  - The tournament format mirrors that of the FIFA World Cup, with a round-robin group stage followed by         │
│  knockout rounds, but without a third-place playoff.                                                            │
│                                                                                                                 │
│  - Notable clubs participating include European giants Chelsea and Paris Saint-Germain (PSG), alongside         │
│  Manchester City and Lionel Messi’s Inter Miami from MLS.                                                       │
│                                                                                                                 │
│  - The knockout phase began on June 28, 2025.                                                                   │
│                                                                                                                 │
│  - Chelsea advanced to the final after defeating Fluminense 2-0 in the semifinals.                              │
│                                                                                                                 │
│  - PSG reached the final by delivering a dominant 4-0 victory over Real Madrid.                                 │
│                                                                                                                 │
│  - The final match of the Club World Cup is scheduled for July 13, 2025, in New Jersey.                         │
│                                                                                                                 │
│  - Broadcast rights are held in China by the streaming platform Migu and in South Korea by Coupang Play.        │
│                                                                                                                 │
│  - The tournament has faced challenges such as a severe heat wave across the US, impacting players and          │
│  coaches.                                                                                                       │
│                                                                                                                 │
│  - An innovative feature introduced this year is that referees are wearing body cameras, and some footage is    │
│  being shared with audiences, providing a new perspective on refereeing.                                        │
│                                                                                                                 │
│  - There was team reshuffling prior to the tournament as Club Leon was removed due to violations of multi-club  │
│  ownership criteria and will be replaced accordingly.  

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8f249a12-4e54-4d74-afe8-7d10eed56cf7                                                                     │
│  Agent: Response Generator                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 40ed7443-2bf9-4a19-b9ca-b9caf3d7280a                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The 2025 FIFA Club World Cup is currently taking place in the United States from June 14 to      │
│  July 13, 2025. This edition of the tournament features a significant change with an expanded format including  │
│  32 clubs competing in 12 venues across 11 US cities. It is designed as a prelude to the 2026 FIFA World Cup.   │
│                                                                                                                 │
│  Key details and happenings in the tournament include:                                                          │
│                                                                                                                 │
│  - The tournament format mirrors that of the FIFA World Cup, with a round-robin group stage followed by         │
│  knockout rounds, but without a third-place playoff.                                                            │
│                                                                                                                 │
│  - Notable clubs participating include European giants Chelsea and Paris Saint-Germain (PSG), alongside         │
│  Manchester City and Lionel Messi’s Inter Miami from MLS.                                                       │
│                                                                                                                 │
│  - The knockout phase began on June 28, 2025.                                                                   │
│                                                                                                                 │
│  - Chelsea advanced to the final after defeating Fluminense 2-0 in the semifinals.                              │
│                                                                                                                 │
│  - PSG reached the final by delivering a dominant 4-0 victory over Real Madrid.                                 │
│                                                                                                                 │
│  - The final match of the Club World Cup is scheduled for July 13, 2025, in New Jersey.                         │
│                                                                                                                 │
│  - Broadcast rights are held in China by the streaming platform Migu and in South Korea by Coupang Play.        │
│                                                                                                                 │
│  - The tournament has faced challenges such as a severe heat wave across the US, impacting players and          │
│  coaches.                                                                                                       │
│                                                                                                                 │
│  - An innovative feature introduced this year is that referees are wearing body cameras, and some footage is    │
│  being shared with audiences, providing a new perspective on refereeing.                                        │
│                                                                                                                 │
│  - There was team reshuffling prior to the tournament 

The 2025 FIFA Club World Cup is currently taking place in the United States from June 14 to July 13, 2025. This edition of the tournament features a significant change with an expanded format including 32 clubs competing in 12 venues across 11 US cities. It is designed as a prelude to the 2026 FIFA World Cup.

Key details and happenings in the tournament include:

- The tournament format mirrors that of the FIFA World Cup, with a round-robin group stage followed by knockout rounds, but without a third-place playoff.

- Notable clubs participating include European giants Chelsea and Paris Saint-Germain (PSG), alongside Manchester City and Lionel Messi’s Inter Miami from MLS.

- The knockout phase began on June 28, 2025.

- Chelsea advanced to the final after defeating Fluminense 2-0 in the semifinals.

- PSG reached the final by delivering a dominant 4-0 victory over Real Madrid.

- The final match of the Club World Cup is scheduled for July 13, 2025, in New Jersey.

- Broadcast rights 

In [ ]:
from crewai import Agent
from crewai_tools import MCPServerAdapter


server_params = [
    {
        "url": "http://localhost:8000/sse",
        "transport": "sse"
    },
    {
    "url": "http://localhost:8080/sse",
    "transport": "sse"
    }
  ]


# Example usage (uncomment and adapt once server_params is set):
with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")

In [ ]:
from crewai import Agent
from crewai_tools import MCPServerAdapter


# 2. SSE Server:
server_params = {
    "url": "http://localhost:8000/sse",
    "transport": "sse"
}

with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")

In [ ]:
from crewai import Agent, Task, Crew

from crewai_tools import MCPServerAdapter

# MCP configuration for websearch and memory MCP servers
server_params = [
    {
        "url": "http://localhost:8000/sse",
        "transport": "sse"
    }
  ]

# Create agents under the MCP context manager
with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")
    # Assistant Agent
    memory_manager = Agent(
        role="Clears memory",
        goal="Clear the memory",
        backstory="I am a helpful AI assistant that can clear the memory.",
        allow_delegation=False,
        tools=[mcp_tools["clear_graph"]]
    )


    memory_task = Task(
        description="Clear the memory",
        agent=memory_manager,
        expected_output="Confirmation of memory cleared"
    )

    # Create crew with all agents and tasks
    crew = Crew(
        agents=[memory_manager],
        tasks=[ memory_task],
        verbose=True
    )

    # Example usage
    result = crew.kickoff(
        inputs={
            
        }
    )
    print(result)


In [ ]:
from crewai import Agent, Task, Crew

from crewai_tools import MCPServerAdapter

# MCP configuration for websearch and memory MCP servers
server_params = [
    {
        "url": "http://localhost:8000/sse",
        "transport": "sse"
    }
  ]

# Create agents under the MCP context manager
with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")